In [1]:
%env CLASSPATH = nzjdbc3.jar


env: CLASSPATH=nzjdbc3.jar


In [28]:
import warnings
warnings.filterwarnings('ignore')
from nzpyida import IdaDataBase, IdaDataFrame
from nzpyida.ae import NZFunGroupedApply,NZFunTApply
import numpy as np

#### Connecting to the database

In [38]:
nzpy_dsn ={
        "database":"housing",
         "port" :5480,
        "host" : "x.xx.xxx.xxx",
        "securityLevel":3,
        "logLevel":0
       
        }

In [39]:
idadb = IdaDataBase(nzpy_dsn, uid="admin", pwd="password")


In [40]:
properties_idadf =  IdaDataFrame(idadb,'properties')
cleaned_properties_idadf = IdaDataFrame(idadb,'cleaned_properties')

In [5]:
properties_idadf.head()


,ID,REGION,TYPE,SQFEET,BEDS,BATHS,CATS_ALLOWED,DOGS_ALLOWED,SMOKING_ALLOWED,WHEELCHAIR_ACCESS,ELECTRIC_VEHICLE_CHARGE,COMES_FURNISHED,LAUNDRY_OPTIONS,PARKING_OPTIONS,lat,lon,PRICE_PREDICTED,accuracy
0,7004529237,florence / muscle shoals,house,815.0,2.0,2.0,1,1,1,0,0,0,w/d in unit,off-street parking,34.757801,-87.697098,606.817973,-0.527060
1,7004588837,tallahassee,house,780.0,2.0,1.0,1,1,1,0,0,0,w/d in unit,off-street parking,30.428600,-84.259300,718.665190,0.738930
2,7004638292,flagstaff / sedona,apartment,814.0,2.0,1.5,1,1,1,0,0,0,w/d in unit,off-street parking,35.212299,-111.578003,1337.036042,0.281961
3,7004645442,columbus,house,700.0,3.0,3.0,1,1,1,0,0,0,w/d in unit,off-street parking,32.561600,-84.855904,622.568214,0.701028
4,7004692207,athens,apartment,1232.0,2.0,2.0,1,1,0,0,0,0,w/d hookups,off-street parking,33.934502,-83.461601,1326.713559,0.656277


In [11]:
properties_idadf.count()

ID                         265144
REGION                     265144
PRICE                      265144
TYPE                       265144
SQFEET                     265144
BEDS                       265144
BATHS                      265144
CATS_ALLOWED               265144
DOGS_ALLOWED               265144
SMOKING_ALLOWED            265144
WHEELCHAIR_ACCESS          265144
ELECTRIC_VEHICLE_CHARGE    265144
COMES_FURNISHED            265144
LAUNDRY_OPTIONS            265144
PARKING_OPTIONS            265144
LAT                        265144
LON                        265144
POINT                      265144
DISTANCE_TO_SCHOOLS        167100
STATE_PARKS_COUNT          265144
dtype: int64

#### Geo-spatial use-cases:

- Finding nearest properties for a selected property of interest

In [25]:
def get_houses_in_the_area(record_id,buffer_miles):
    try: 

        #User selects a record  

        record_selected_by_user = idadb.ida_query(f'''SELECT inza..ST_AsText(POINT) as POINT,REGION,PRICE from cleaned_properties where id = {record_id}''') 

        
        

        #Getting the buffer region around the specified point  

        buffer_region = idadb.ida_query(f'''SELECT inza..ST_AsText(inza..ST_Buffer(inza..ST_WKTToSQL('{record_selected_by_user['POINT'][0]}'),{buffer_miles},8,'mile'))''')  

        

        #Properties within the buffer region 

        properties_within_distance = idadb.ida_query(f'''SELECT price from cleaned_properties where inza..ST_Contains( inza..ST_WKTToSQL('{buffer_region[0]}'),inza..ST_WKTToSQL(inza..ST_AsText(cleaned_properties.point)));''')  

        

        summary_list = [max(properties_within_distance),min(properties_within_distance),np.mean(properties_within_distance)] 

        

        properties_list = idadb.ida_query(f'''SELECT * from 		cleaned_properties where 

        region='{record_selected_by_user['REGION'][0]}'  

        and PRICE BETWEEN {min(properties_within_distance)} and { 

        max(properties_within_distance)} ''') 

        

        return summary_list,properties_list 

    except: 

        return 'Statistics for the region could not be found' 

In [27]:
summary_list , properties_list = get_houses_in_the_area(7004588837,10)

##### Statistics:

In [29]:
if summary_list[0] != None or summary_list[1] !=None or summary_list[2] != None:
    print(f"Maximum price : {summary_list[0]}$")
    print(f"Minimum price : {summary_list[1]}$")
    print(f"Average price : {(summary_list[2])}$")
else:
    print('Summary information could not be found..')

Maximum price : 3495.0$
Minimum price : 300.0$
Average price : 952.1256627783669$


In [30]:
properties_list

,ID,REGION,PRICE,TYPE,SQFEET,BEDS,BATHS,CATS_ALLOWED,DOGS_ALLOWED,SMOKING_ALLOWED,WHEELCHAIR_ACCESS,ELECTRIC_VEHICLE_CHARGE,COMES_FURNISHED,LAUNDRY_OPTIONS,PARKING_OPTIONS,LAT,LON,POINT,DISTANCE_TO_SCHOOLS,STATE_PARKS_COUNT
0,7046229774,tallahassee,799.0,apartment,954.0,2.0,2.0,1,1,1,0,0,0,w/d hookups,off-street parking,30.4195,-84.234901,     UÀ    UÀ   `dk>@   `dk>@æ      ...,0.679534,0
1,7046229367,tallahassee,799.0,apartment,954.0,2.0,2.0,1,1,1,0,0,0,w/d hookups,off-street parking,30.4195,-84.234901,     UÀ    UÀ   `dk>@   `dk>@æ      ...,0.679534,0
2,7023939868,tallahassee,799.0,apartment,954.0,2.0,2.0,1,1,1,0,0,0,w/d hookups,off-street parking,30.4195,-84.234901,     UÀ    UÀ   `dk>@   `dk>@æ      ...,0.679534,0
3,7023936944,tallahassee,799.0,apartment,954.0,2.0,2.0,1,1,1,0,0,0,w/d hookups,off-street parking,30.4195,-84.234901,     UÀ    UÀ   `dk>@   `dk>@æ      ...,0.679534,0
4,7023628504,tallahassee,799.0,apartment,954.0,2.0,2.0,1,1,1,0,0,0,w/d hookups,off-street parking,30.4195,-84.234901,     UÀ    UÀ   `dk>@   `dk>@æ      ...,0.679534,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973,7009345317,tallahassee,799.0,apartment,954.0,2.0,2.0,1,1,1,0,0,0,w/d hookups,off-street parking,30.4195,-84.234901,     UÀ    UÀ   `dk>@   `dk>@æ      ...,0.679534,0
1974,7009342601,tallahassee,799.0,apartment,954.0,2.0,2.0,1,1,1,0,0,0,w/d hookups,off-street parking,30.4195,-84.234901,     UÀ    UÀ   `dk>@   `dk>@æ      ...,0.679534,0
1975,7006011244,tallahassee,795.0,duplex,850.0,2.0,2.0,0,0,0,0,0,0,w/d in unit,off-street parking,30.4188,-84.245102,    À¯UÀ   À¯UÀ   6k>@   6k>@æ      ...,0.844399,0
1976,7023613842,tallahassee,879.0,apartment,962.0,1.0,1.0,0,0,1,0,0,0,w/d in unit,off-street parking,30.4874,-84.318901,    àhUÀ   àhUÀ   @Æ|>@   @Æ|>@æ      ...,0.427583,0


- Finding the cheaper rental options closer to the selected property 

In [39]:
def get_discounted_houses(record_id): 

    try: 

#User selects a record  

        record_selected_by_user = idadb.ida_query(f'''SELECT inza..ST_AsText(POINT) as POINT,REGION, 

        PRICE from cleaned_properties where id = {record_id}''') 

        
        

        #Get all the records where it is lower than the discounted price. 

        selected_house_price = record_selected_by_user['PRICE'][0] 

        discount = 10 

        
        

        final_price = selected_house_price*(1-(discount/100)) 

        
        

        #Calculate distances between properties and display all the records matching the 	#condition 

        
        

        final_houses_with_discounted_prices = idadb.ida_query(f'''select temp.id,temp.price,temp.distance_between_properties,temp.region,
        temp.beds,temp.baths,temp.sqfeet,temp.cats_allowed,temp.dogs_allowed,temp.smoking_allowed,temp.wheelchair_access,temp.electric_vehicle_charge,
        temp.comes_furnished
         from  ( select cleaned_properties.*,inza..ST_Distance(
                            inza..ST_Transform(inza..ST_WKTToSQL(inza..ST_AsText(cleaned_properties.point)), 4326), 
                            inza..ST_Transform(inza..ST_WKTToSQL('{record_selected_by_user['POINT'][0]}'),4326), 'mile')as 
                            distance_between_properties from cleaned_properties) as temp 
                            where temp.price<={final_price} and temp.distance_between_properties<5 order by temp.distance_between_properties;''')

        

        return final_houses_with_discounted_prices 

 

    except: 

        return 'No houses found nearby at a discount' 

In [40]:
discounted_houses = get_discounted_houses(7004588837)
discounted_houses

,ID,PRICE,DISTANCE_BETWEEN_PROPERTIES,REGION,BEDS,BATHS,SQFEET,CATS_ALLOWED,DOGS_ALLOWED,SMOKING_ALLOWED,WHEELCHAIR_ACCESS,ELECTRIC_VEHICLE_CHARGE,COMES_FURNISHED
0,7045946902,395.0,2.768783,tallahassee,3.0,0.0,1500.0,0,0,0,0,0,0
1,7045959216,415.0,2.768783,tallahassee,4.0,0.0,1650.0,0,0,0,0,0,0
2,7045950765,410.0,2.768783,tallahassee,4.0,0.0,1500.0,0,0,0,0,0,0
3,7045936341,450.0,3.589566,tallahassee,2.0,0.0,1250.0,0,0,0,0,0,0
4,7023838406,395.0,3.918576,tallahassee,3.0,2.0,929.0,1,1,1,0,0,1
5,7045964077,415.0,3.918576,tallahassee,5.0,3.0,2200.0,0,0,0,0,0,0
6,7044742622,395.0,3.942118,tallahassee,3.0,2.5,1650.0,0,0,0,0,0,0
7,7041234093,375.0,3.944454,tallahassee,3.0,2.0,1209.0,1,1,1,0,0,0
8,7040004702,390.0,3.944454,tallahassee,4.0,2.0,1516.0,1,1,1,0,0,0
9,7039997105,415.0,3.944454,tallahassee,3.0,2.0,1209.0,1,1,1,0,0,0


- Finding the green space around the selected property 

In [39]:
#Updating the park count for each record
idadb.ida_query(f'''update cleaned_properties set cleaned_properties.state_parks_count = parks_count.count from 
(select temp.id,count(temp.c) as count from (select cleaned_properties.id,inza..ST_Intersects( 
inza..ST_WKTToSQL(inza..ST_AsText( 
inza..ST_Buffer(cleaned_properties.point 
,100,8,'mile'))) 
, national_parks.the_geom) as c from cleaned_properties,national_parks where c='True') as temp group by temp.id) as  
parks_count where parks_count.id=cleaned_properties.id;''')

In [40]:
#Get the national_parks_area for each record 

idadb.ida_query(f'''update green_area set green_area.national_parks_area = d.sum from(  

select f.id,SUM(f.area) as sum from (select temp.id,inza..ST_Area(inza..ST_WKTToSQL(temp.c),'mile') as area  

from (select cleaned_properties.id,inza..ST_AsText(inza..ST_Intersection(  

inza..ST_WKTToSQL(inza..ST_AsText(  

inza..ST_Buffer(cleaned_properties.point ,100,8,'mile')))  

, national_parks.the_geom)) as c from cleaned_properties,national_parks ) as temp  

where temp.c!='POINT EMPTY') as f group by f.id) as d where d.id = green_area.id;''') 

In [45]:
def get_green_area(record_id):
    try:
        record_selected_by_user =  cleaned_properties_idadf.ida_query(f'''SELECT inza..ST_AsText(POINT) as POINT,REGION from cleaned_properties where id = {record_id} ''')
        
        #Getting the properties in the same region ordered by national_parks_area and park_count
        ranked_properties = cleaned_properties_idadf.ida_query(f'''select green_area.national_parks_area,temp.state_parks_count,temp.ID,
        temp.PRICE,temp.REGION,temp.BEDS,temp.BATHS,temp.SQFEET,temp.CATS_ALLOWED,temp.DOGS_ALLOWED, temp.SMOKING_ALLOWED,temp.WHEELCHAIR_ACCESS, 
        temp.ELECTRIC_VEHICLE_CHARGE,temp.COMES_FURNISHED,temp.LAT as "lat" , temp.LON as "lon"  from (SELECT * from cleaned_properties where 
                                        cleaned_properties.region ='{record_selected_by_user['REGION'][0]}') as temp inner join green_area
                                        on green_area.id = temp.id 
                                        order by state_parks_count desc,national_parks_area desc;''')

        green_information = [200,ranked_properties]

        return green_information
    except:
        return [404,'Unknown error happened! Green information could not be found, please try again..']

In [46]:
green_information = get_green_area(7043564878)
green_information[1]

,NATIONAL_PARKS_AREA,STATE_PARKS_COUNT,ID,PRICE,REGION,BEDS,BATHS,SQFEET,CATS_ALLOWED,DOGS_ALLOWED,SMOKING_ALLOWED,WHEELCHAIR_ACCESS,ELECTRIC_VEHICLE_CHARGE,COMES_FURNISHED,lat,lon
0,6272.849397,4,7043795019,975.0,reno / tahoe,3.0,2.5,1819.0,1,0,1,0,0,0,36.068100,-115.079002
1,435.330687,2,7048719596,988.0,reno / tahoe,3.0,2.5,1400.0,1,1,1,0,0,0,34.231300,-119.045998
2,435.330687,2,7044091106,988.0,reno / tahoe,3.0,2.5,1400.0,1,1,1,0,0,0,34.231300,-119.045998
3,435.330687,2,7044091322,988.0,reno / tahoe,3.0,2.5,1400.0,1,1,1,0,0,0,34.231300,-119.045998
4,435.330687,2,7048019861,988.0,reno / tahoe,3.0,2.5,1400.0,1,1,1,0,0,0,34.231300,-119.045998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,0.000000,0,7041479328,1890.0,reno / tahoe,3.0,2.5,2413.0,0,1,1,0,0,0,39.590000,-119.150002
2532,0.000000,0,7039350178,1100.0,reno / tahoe,2.0,2.0,1400.0,1,1,1,0,0,0,39.648102,-119.292000
2533,0.000000,0,7044292392,1100.0,reno / tahoe,2.0,2.0,1425.0,1,1,1,0,0,0,39.648102,-119.292000
2534,0.000000,0,7049031613,1075.0,reno / tahoe,1.0,1.0,790.0,1,0,1,0,0,0,0.000000,0.000000


- Predicting the house prices using the geospatial features  

In [49]:
#Adding distance_to_schools_column
idadb.ida_query('''update cleaned_properties set cleaned_properties.distance_to_schools = temp.distance_to_schools  

from (select cleaned_properties.id,MIN(inza..ST_DISTANCE( 

inza..ST_Transform(cleaned_properties.point, 4326),  

inza..ST_Transform(Public_Schools.point,4326), 

'mile')) as distance_to_schools from Public_Schools,cleaned_properties  

where cleaned_properties.REGION = LOWER(Public_Schools.CITY) 

GROUP BY cleaned_properties.id) as temp 

where temp.id = cleaned_properties.id; ''') 

##### In-database function:

In [41]:
if idadb.exists_table("predictions"):
        idadb.drop_table("predictions")


code_house_prediction_model = """def house_prediction_model(self,df):
                import numpy as np
                import pandas as pd
                
                import sklearn
                import pickle
                import subprocess
                
                from sklearn.model_selection import cross_val_score
                from sklearn.model_selection import train_test_split
                from sklearn.linear_model import LinearRegression
                from sklearn.ensemble import GradientBoostingRegressor
                # from xgboost import XGBRegressor
                from sklearn.preprocessing import LabelEncoder
                from sklearn.metrics import r2_score,mean_squared_error
                
            
                
                data = df.copy()
                
                

                data['DISTANCE_TO_SCHOOLS'] = data['DISTANCE_TO_SCHOOLS'].replace(to_replace = 0 , value = 10000)
                data['DISTANCE_TO_SCHOOLS'] = data['DISTANCE_TO_SCHOOLS']/data['DISTANCE_TO_SCHOOLS'].abs().max()
            
                encoder={}
                categorical_columns_list = []
                for i in data.columns:
                    if data[i].dtype == 'object':
                        categorical_columns_list.append(i)
                        encoder[i] = LabelEncoder()
                        encoder[i].fit(list(data[i].values))
                        data[i] = encoder[i].transform(list(data[i].values))
                
                
                #Splitting into training and testing dataset
                
                y = data['PRICE']
                X = data.drop(columns=['PRICE'],axis=1)
                
                X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20 , train_size = 0.80, random_state=100)
                
        
                
                #Gradient Boosting Regression
                from sklearn.ensemble import GradientBoostingRegressor
                
                gbr = GradientBoostingRegressor()
                gbr.fit(X_train, y_train)
                y_pred_gbr= gbr.predict(X_test)
                y_pred_gbr = np.around(y_pred_gbr,decimals=0)
                r2_score = gbr.score(X_test,y_test)
                
                
                from sklearn.model_selection import cross_val_score
                cv_scores = np.sqrt(-cross_val_score(gbr, X, y,scoring='neg_mean_squared_error',cv=2))
                final_score = (np.mean(cv_scores/y.max()))
                
                predictions = X_test.copy()
                
                for column in categorical_columns_list:   
                    predictions[column] = encoder[column].inverse_transform(list(predictions[column].values))
                
                predictions['PRICE_PREDICTED'] = y_pred_gbr
                predictions['ACCURACY'] = r2_score
                def print_output(x):
                    row = [x['ID'],x['REGION'],
                   x['TYPE'],x['PRICE_PREDICTED'],
                    x['SQFEET'],x['BEDS'],
                    x['BATHS'],x['CATS_ALLOWED'],x['DOGS_ALLOWED'],x['SMOKING_ALLOWED'],
                    x['WHEELCHAIR_ACCESS'],x['ELECTRIC_VEHICLE_CHARGE'],
                    x['COMES_FURNISHED'],x['LAUNDRY_OPTIONS'],x['PARKING_OPTIONS'],x['DISTANCE_TO_SCHOOLS'],x['ACCURACY']]
                    self.output(row)
                predictions.apply(print_output, axis=1)
                
            
                pickle.dump(gbr, open('/export/home/nz/git_repo/models-repo/model.pkl','wb'))
                
                    
"""

In [42]:
output_signature = {'ID':'int64','REGION':'str','TYPE':'str','PRICE_PREDICTED':'double',
    'SQFEET':'double','BEDS':'float','BATHS':'float','CATS_ALLOWED':'int','DOGS_ALLOWED':'int',
'SMOKING_ALLOWED':'int','WHEELCHAIR_ACCESS':'int','ELECTRIC_VEHICLE_CHARGE'	: 'int','COMES_FURNISHED':'int',
'LAUNDRY_OPTIONS':'str','PARKING_OPTIONS':'str','DISTANCE_TO_SCHOOLS':'double','ACCURACY':'double'
    }


In [43]:
nz_fun_grouped_apply = NZFunGroupedApply(df=cleaned_properties_idadf, columns = ['ID','REGION','TYPE','SQFEET','PRICE','BEDS','BATHS','CATS_ALLOWED','DOGS_ALLOWED'
,'SMOKING_ALLOWED','WHEELCHAIR_ACCESS','ELECTRIC_VEHICLE_CHARGE','COMES_FURNISHED','LAUNDRY_OPTIONS','PARKING_OPTIONS','DISTANCE_TO_SCHOOLS'],code_str=code_house_prediction_model, index='REGION',fun_name ="house_prediction_model", output_table='predictions',output_signature=output_signature)
result_idadf = nz_fun_grouped_apply.get_result()
result = result_idadf.as_dataframe()
print(result)

               ID       REGION          TYPE  PRICE_PREDICTED  SQFEET  BEDS  \
0      7049993823  SF bay area     apartment           3001.0  1073.0   2.0   
1      7050001407  SF bay area     apartment           3572.0  1650.0   2.0   
2      7049954093  SF bay area     apartment           2000.0   640.0   1.0   
3      7037717802  SF bay area     apartment           3213.0  1000.0   1.0   
4      7049934118  SF bay area     apartment           2480.0   660.0   1.0   
...           ...          ...           ...              ...     ...   ...   
53141  7049040991         yuma     apartment            825.0   885.0   2.0   
53142  7040443337         yuma  manufactured            790.0   980.0   2.0   
53143  7028129768         yuma     apartment            848.0   885.0   2.0   
53144  7037956523         yuma     apartment            904.0   845.0   2.0   
53145  7035915874         yuma     apartment            791.0   675.0   2.0   

       BATHS  CATS_ALLOWED  DOGS_ALLOWED  SMOKING_A